In [28]:
# Read environment variables

import os
import traceback

import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

openai_url      = os.environ['AZURE_OPENAI_URL']
openai_key      = os.environ['AZURE_OPENAI_KEY']
openai_version  = os.environ['AZURE_OPENAI_API_VERSION']  # "2023-07-01-preview" 
embeddings_dep  = os.environ['AZURE_OPENAI_EMBEDDINGS_DEP']
completions_dep = os.environ['AZURE_OPENAI_COMPLETIONS_DEP']

openai.api_type = "azure"
openai.api_version = openai_version
openai.api_base = openai_url
openai.api_key  = openai_key

print("openai_url:      {}".format(openai_url))
print("openai_key:      {}".format(openai_key[0:3]))
print("openai_version:  {}".format(openai_version))
print("embeddings_dep:  {}".format(embeddings_dep))
print("completions_dep: {}".format(completions_dep))

try:
    aoai_client = AzureOpenAI(
        azure_endpoint=openai_url,
        api_key=openai_key,
        api_version=openai_version,
    )
    print("aoai_client created")
except Exception as e:
    print("Exception while creating AzureOpenAI client: {}".format(str(e)))
    print(traceback.format_exc())


openai_url:      https://gbbcjcaigopenai3.openai.azure.com/
openai_key:      d1b
openai_version:  2023-07-01-preview
embeddings_dep:  embeddings
completions_dep: gpt4
aoai_client created


In [29]:
# Generate embeddings for a given string value.

import traceback
from pysrc.util.fs import FS

try:
    text = "A long time ago came a man on a track. Walking thirty miles with a sack on his back."
    response = aoai_client.embeddings.create(input=text, model=embeddings_dep)
    embedding = response.data[0].embedding

    print("embedding length: {}".format(len(embedding)))   
    print("embedding[:4]: {}".format(embedding[:4])) 
    FS.write_json(embedding, "tmp/embedding.json")
except Exception as e:
    print("Exception while creating embedding: {}".format(str(e)))
    print(traceback.format_exc())


embedding length: 1536
embedding[:4]: [0.018863078206777573, -0.006897789891809225, 0.015606330707669258, 0.007549139205366373]


In [30]:
# Generate a simple completion 

import traceback

try:
    response = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Knock knock."},
            {"role": "assistant", "content": "Who's there?"},
            {"role": "user", "content": "Orange."}
        ],
        temperature=0
    )
    print(response)
    print("role: {} -> {}".format(response.choices[0].message.role, response.choices[0].message.content))
    
    response = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capitol of North Carolina?"},
        ],
        temperature=0
    )
    print(response)
    print("role: {} -> {}".format(response.choices[0].message.role, response.choices[0].message.content))
    
except Exception as e:
    print("Exception while creating completion: {}".format(str(e)))
    print(traceback.format_exc())

ChatCompletion(id='chatcmpl-9T8TvTfB7wUpJarAggGazcNBkwSHA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Orange who?', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1716731143, model='gpt-4', object='chat.completion', system_fingerprint='fp_7ec89fabc6', usage=CompletionUsage(completion_tokens=3, prompt_tokens=35, total_tokens=38), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
role: assistant -> Orange who?
ChatCompletion(id='chatcmpl-9T8Tv3th7zdddlxnNVzE